## Planetary Ensemble Prediction and Error Analysis

This notebook uses the saved MLP ensemble models and the fitted StandardScaler to generate predictions for a specified time period and planet. It then calculates and visualizes the prediction errors.

In [1]:
# --- 1. SETUP AND IMPORTS ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime, timedelta
import os
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


# Import your utility file (ensure stars_utils.py is in the same directory)
import stars_utils

print(f'TensorFlow Version: {tf.version}')

TensorFlow Version: <module 'tensorflow._api.v2.version' from '/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tensorflow/_api/v2/version/__init__.py'>


In [2]:
# --- 2. CONFIGURATION ---

# Prediction Target and Timeframe
TARGET_PLANET = 'mars' # Change this to 'jupiter', 'venus', etc. for analysis
START_DATE = datetime(1970, 1, 1)
END_DATE = datetime(2025, 1, 1)
TIME_STEP = timedelta(days=1) # Weekly data for prediction

# Model and Scaler Locations
MODEL_DIR = 'models' 
MODELS_LIS = ['mm0', 'mm1', 'mm2'] 
SCALER_FILEPATH = os.path.join(MODEL_DIR, f'feature_scaler_{TARGET_PLANET}.pkl') 


# Create model directory if it doesn't exist (useful for saving)
os.makedirs(MODEL_DIR, exist_ok=True)
print(f'Configuration loaded. Target Planet: {TARGET_PLANET.capitalize()}')

Configuration loaded. Target Planet: Mars


In [3]:
# --- 3. LOAD SCALER AND MODELS ---

print('Loading assets...')

# # Load the fitted scaler object
scaler = stars_utils.load_scaler(SCALER_FILEPATH)

# if scaler is None:
#     raise FileNotFoundError('FATAL: Scaler not loaded. Run your training script first to save it!')

# Load the ensemble models
ensemble_models = stars_utils.models_loader(MODEL_DIR, MODELS_LIS, TARGET_PLANET)

if not ensemble_models:
    raise ValueError('FATAL: Failed to load any models. Check MODEL_DIR and MODELS_LIS.')

print(f'Successfully loaded {len(ensemble_models)} models and the feature scaler.')

Loading assets...
Scaler loaded successfully from: models/feature_scaler_mars.pkl
Attempting to load 3 models for mars from 'models'...
Attempting to load: models/mars_position_predictor_mm0.keras
Attempting to load: models/mars_position_predictor_mm1.keras
File not found at models/mars_position_predictor_mm2.keras. Falling back to: models/mars_position_predictor_mm2.h5
Successfully loaded 3 models.
Successfully loaded 3 models and the feature scaler.


In [4]:
# --- 4. GENERATE GROUND TRUTH AND PREPARE INPUT ---

print(f'Generating ground truth data for {TARGET_PLANET.capitalize()}...')

# Generate the Ground Truth data (actual positions)
df_planet = stars_utils.generate_planetary_ephemeris_df(
    target_planet=TARGET_PLANET, 
    start_date=START_DATE, 
    end_date=END_DATE, 
    time_step=TIME_STEP
)

df_planet = stars_utils.add_astronomy_features(df_planet, TARGET_PLANET)

FEATURES = [
        'Time_Index', 'Time_Index_2','Time_Index_3',
        'Sin_Year', 'Cos_Year',
        'Sin_Mars', 'Cos_Mars', 
        'Sin_Synodic', 'Cos_Synodic'
]
TARGETS = ['X_au', 'Y_au', 'Z_au']

# Extract the feature (Julian Date)
X_predict = df_planet[FEATURES]

# Use the loaded scaler to TRANSFORM the prediction data
X_predict_scaled = scaler.transform(X_predict)

# Extract the ground truth Cartesian coordinates
y_ground_truth = df_planet[TARGETS].values
print(f'Data generated and scaled. Ready for prediction over {len(df_planet)} steps.')

Generating ground truth data for Mars...
Dataset for Mars created successfully with 20090 data points.
Calculated mars's Synodic Period with Earth: 779.91 days.
Added dynamic features (Time Index, Polynomial, Earth Cycle, Target Cycle, Synodic Cycle, Interaction) to the DataFrame.
Data generated and scaled. Ready for prediction over 20090 steps.


In [5]:
X_predict_scaled.shape

(20090, 9)

In [6]:
# --- 5. GENERATE ENSEMBLE PREDICTIONS ---

print('Generating Ensemble Predictions...')
y_pred_list = []

for i, model in enumerate(ensemble_models):
    # Predict the XYZ coorinates (3 targets)
    y_pred = model.predict(X_predict_scaled, verbose=1) # Suppress prediction output
    y_pred_list.append(y_pred)

# Ensemble prediction is the simple average of all SCALED model outputs
y_ensemble_pred_au = np.mean(y_pred_list, axis=0)

# # CRITICAL: Descale the final ensemble prediction back to AU units (X, Y, Z)
# y_ensemble_pred_au = scaler.inverse_transform(y_ensemble_pred_scaled)

# --- 6. AUGMENT DATAFRAME WITH RESULTS ---

# Add Prediction columns
df_planet['X_pred'] = y_ensemble_pred_au[:, 0]
df_planet['Y_pred'] = y_ensemble_pred_au[:, 1]
df_planet['Z_pred'] = y_ensemble_pred_au[:, 2]

# Add Residual (Error) columns: Actual - Predicted
df_planet['X_Error'] = df_planet['X_au'] - df_planet['X_pred']
df_planet['Y_Error'] = df_planet['Y_au'] - df_planet['Y_pred']
df_planet['Z_Error'] = df_planet['Z_au'] - df_planet['Z_pred']

print('Prediction complete. DataFrame augmented with prediction results.')

Generating Ensemble Predictions...
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 249us/step
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 246us/step
628/628 ━━━━━━━━━━━━━━━━━━━━ 0s 249us/step
Prediction complete. DataFrame augmented with prediction results.


In [7]:
# --- 7. ANALYSIS: ERROR SUMMARY ---

print("\n4. Final Analysis and Error Summary")
print(f"Prediction Period: {START_DATE.year} to {END_DATE.year}")

# Calculate the TRUE 3D Position RMSE (Euclidean Distance RMSE)
squared_error_3d = np.sum((y_ground_truth - y_ensemble_pred_au)**2, axis=1)
rmse_3d_position = np.sqrt(np.mean(squared_error_3d))

# Get Angular Error (which is the 3D position error magnitude)
df_planet['Angular_Error_AU'] = stars_utils.get_angular_error_arcsec(df_planet, stars_utils.ts) 

df_planet['Pos_Error_AU'] = df_planet['Angular_Error_AU']
max_error_au = df_planet['Pos_Error_AU'].max()
mean_error_au = df_planet['Pos_Error_AU'].mean()


print(f"\nTRUE 3D Position RMSE: {rmse_3d_position:.6f} AU")
print(f"Max 3D Position Error: {max_error_au:.6f} AU")
print(f"Mean 3D Position Error: {mean_error_au:.6f} AU")

print("\n--- Top 5 rows with Predictions and Errors ---")
print(df_planet[['Time_UTC', 'X_au', 'X_pred', 'Pos_Error_AU', 'Y_au', 'Y_pred']].head())

# --- 7. VISUALIZATION IDEAS ---
print("\n5. Visualization Ideas (using matplotlib/Skyfield):")
print("1. 3D Position Error Over Time (Plotting 'Pos_Error_AU' vs 'Time_UTC').")
print("2. Individual Coordinate Errors Over Time (Plotting 'X_Error', 'Y_Error', 'Z_Error').")

# df_planet[['Time_UTC', 'RA_deg', 'RA_pred', 'RA_Error', 'Distance_AU', 'Distance_pred']].head()


4. Final Analysis and Error Summary
Prediction Period: 1970 to 2025
Calculating 3D positional error magnitude...

TRUE 3D Position RMSE: 0.211103 AU
Max 3D Position Error: 0.351218 AU
Mean 3D Position Error: 0.194793 AU

--- Top 5 rows with Predictions and Errors ---
    Time_UTC      X_au    X_pred  Pos_Error_AU      Y_au    Y_pred
0 1970-01-01  1.506684  1.449995      0.110176 -0.422805 -0.428556
1 1970-01-02  1.519418  1.461625      0.109124 -0.406782 -0.413553
2 1970-01-03  1.531953  1.473093      0.108075 -0.390533 -0.398331
3 1970-01-04  1.544287  1.484405      0.107017 -0.374060 -0.382884
4 1970-01-05  1.556413  1.495556      0.105953 -0.357364 -0.367215

5. Visualization Ideas (using matplotlib/Skyfield):
1. 3D Position Error Over Time (Plotting 'Pos_Error_AU' vs 'Time_UTC').
2. Individual Coordinate Errors Over Time (Plotting 'X_Error', 'Y_Error', 'Z_Error').


In [9]:
stars_utils.plot_error_over_time(df_planet, TARGET_PLANET,
                     error_col='Pos_Error_AU', 
                     title='3D Position Error Over Time', 
                     ylabel='Error (AU)')

# 2. Individual Coordinate Errors Over Time 
# We'll plot X, Y, Z Errors on one graph for comparison
plt.figure(figsize=(14, 6))
plt.plot(df_planet['Time_UTC'], df_planet['X_Error'], label='X Error', linestyle='--')
plt.plot(df_planet['Time_UTC'], df_planet['Y_Error'], label='Y Error', linestyle='-')
plt.plot(df_planet['Time_UTC'], df_planet['Z_Error'], label='Z Error', linestyle=':')
plt.title(f'{TARGET_PLANET.capitalize()} Individual Cartesian Errors Over Time', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Error (AU)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()

TypeError: 'module' object is not callable